In [4]:
import requests
from config import DATA_DOWNLOAD_URL
import pandas as pd
import io
import gzip

In [2]:
df = pd.DataFrame(columns=["DATE", "DEP", "RR", "TN", "TX", "TM"])

In [ ]:
for i in range(95):
    print(f"{i+1:02d}")
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_previous-1950-2023_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp = tmp[tmp["DATE"] >= "1979-01-01"].reset_index(drop=True) #filter by date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])


In [3]:
for i in range(95):
    print(f"{i+1:02d}")
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_latest-2024-2025_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp = tmp[tmp["DATE"] >= "2024-09-01"].reset_index(drop=True) #filter by date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])

01


C:\Users\alexl\AppData\Local\Temp\ipykernel_3440\1035153152.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, tmp])


02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95


In [4]:
df = df.reset_index(drop=True)

In [5]:
df.to_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-current.csv")

In [ ]:
df.to_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-1950-2023.csv")

Make a mean file for each month, each departement for the last 20 years from 2003 to 2023

In [6]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-1950-2023.csv")
df = df[df["DATE"] >= "2003-01-01"]
df = df.dropna()
df = df.drop(columns=["Unnamed: 0"])

In [7]:
dailyDepMean = df.groupby(["DEP", "DATE"]).mean().reset_index()
dailyDepMean['YEAR-MONTH'] = pd.to_datetime(dailyDepMean["DATE"]).dt.to_period('M') #new column with YYYY-MM format

In [9]:
yearlyPrecip = dailyDepMean.groupby(["DEP", "YEAR-MONTH"]).sum()["RR"].reset_index()
yearlyPrecip['MONTH'] = yearlyPrecip['YEAR-MONTH'].dt.month
monthlyPrecip = yearlyPrecip.groupby(["MONTH", "DEP"]).mean()['RR'].reset_index()

In [11]:
yearlyTemp = dailyDepMean[["DEP", "YEAR-MONTH", "TN", "TX", "TM"]].groupby(["DEP", "YEAR-MONTH"]).mean()[["TN", "TX", "TM"]].reset_index()
yearlyTemp['MONTH'] = yearlyTemp['YEAR-MONTH'].dt.month
monthlyTemp = yearlyTemp.groupby(["MONTH", "DEP"]).mean()[["TN", "TX", "TM"]].reset_index()

In [14]:
meteoFranceMean = pd.merge(monthlyPrecip, monthlyTemp, on=["MONTH", "DEP"], how='inner')
meteoFranceMean

,MONTH,DEP,RR,TN,TX,TM
0,1,1,111.142766,-1.012066,5.468150,1.859154
1,1,2,63.543750,1.204320,6.098548,3.519510
2,1,3,60.424884,0.512403,6.722931,3.370427
3,1,4,49.107585,-3.260030,8.497296,1.471727
4,1,5,53.761693,-4.779385,6.117791,-0.398876
...,...,...,...,...,...,...
1135,12,91,62.340454,2.318371,7.686998,4.862220
1136,12,92,46.580000,4.584866,9.176586,6.604274
1137,12,93,44.542857,4.257143,8.658894,6.340369
1138,12,94,64.388095,3.248771,8.242857,5.538479
